In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
Conn=wrds.Connection(wrds_username='')

/Users/vincent/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Loading library list...
Done


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
                                ###Data Download directly from Audit Analytics###
audit_fee=pd.DataFrame()
for year in range(2000,2023):
    audit_fee1=pd.read_csv('/Users/vincent/Desktop/Bentley PHD/Audit Data/Audit fee/audit-data-'+str(year)+'.csv')
    audit_fee=pd.concat([audit_fee,audit_fee1])
audit_fee=audit_fee[audit_fee['Audit Fees ($)'].notnull()]
audit_fee=audit_fee[['CIK Code','Auditor Key','Auditor','Auditor Affiliate','Year Ended Date',\
                     'Audit Fees ($)','Total Non Audit Fees ($)','Total Fees ($)']].\
            drop_duplicates(['CIK Code','Year Ended Date'],keep='first')

###adjust date
audit_fee['Year Ended Date']=pd.to_datetime(audit_fee['Year Ended Date'])
audit_fee['year']=audit_fee['Year Ended Date'].dt.year
audit_fee['month']=audit_fee['Year Ended Date'].dt.month
audit_fee['day']=audit_fee['Year Ended Date'].dt.day
audit_fee['date_adjustment']=0
audit_fee.loc[(audit_fee['month']==1) & (audit_fee['day']<=15), 'date_adjustment']=1
audit_fee.loc[(audit_fee['month']==1) & (audit_fee['day']<=15), 'month']=12
audit_fee['year']=audit_fee['year']-audit_fee['date_adjustment']
audit_fee['fye']=pd.to_datetime(audit_fee[['year','month','day']])
audit_fee['fye']=audit_fee['fye']+MonthEnd(0)
audit_fee['Year Ended Date']=audit_fee['Year Ended Date']+MonthEnd(0)
audit_fee

,CIK Code,Auditor Key,Auditor,Auditor Affiliate,Year Ended Date,Audit Fees ($),Total Non Audit Fees ($),Total Fees ($),year,month,day,date_adjustment,fye
0,823768.0,5.0,Arthur Andersen LLP,NaN,2000-12-31,"48,000,000","31,000,000","79,000,000",2000,12,31,0,2000-12-31
1,313807.0,2.0,Ernst & Young LLP,Ernst & Young LLP,2000-12-31,"28,000,000","51,000,000","79,000,000",2000,12,31,0,2000-12-31
2,831001.0,4.0,KPMG LLP,NaN,2000-12-31,"26,130,000","24,628,000","50,758,000",2000,12,31,0,2000-12-31
3,1089113.0,4.0,KPMG LLP,KPMG Audit Plc,2000-12-31,"25,800,000","15,000,000","40,800,000",2000,12,31,0,2000-12-31
4,1024401.0,5.0,Arthur Andersen LLP,NaN,2000-12-31,"25,000,000","27,000,000","52,000,000",2000,12,31,0,2000-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827,1871890.0,31173.0,Jack Shama CPA,NaN,2022-07-31,"2,000","3,800","5,800",2022,7,31,0,2022-07-31
1828,1842167.0,31173.0,Jack Shama CPA,NaN,2022-07-31,"2,000","2,000","4,000",2022,7,31,0,2022-07-31
1829,1860173.0,31173.0,Jack Shama CPA,NaN,2022-07-31,"2,000","1,750","3,750",2022,7,31,0,2022-07-31
1837,1496998.0,1.0,PricewaterhouseCoopers LLP,NaN,2022-08-31,0,"8,000","8,000",2022,8,31,0,2022-08-31


In [3]:
                                          #  Compustat Fundemental #
###Data from compustat Fundemental Anual
#IB:income before extraordinary items
#oancf: operating  cash flow/ net change in cash from all items classified in the Operating Activities section on a Statement of Cash Flows
#at: Assets - Total
#LT: total liability
#REVT: total revenue
#PPEGT:gross property plant and equipment 
#CSHO:all common shares outstanding at year-end, excluding treasury shares and scrip

#oiadp: Operating Income After Depreciation
#FOPT :  Item 110 (working capital from operations)   only 3800 obs???    WDP 0???
#act: Current Assets - TotalItem 4 (current assets) 
#lct: Current Liabilities - Total Item 5 (current liabilities) 
#che: cash/cash equivalents change in Item 1 (cash)
#dlc: Debt in Current Liabilities change In Item 34 (debt included in current liabilities)
#txp: Income Taxes Payable
#dp: Depreciation and Amortization
#AU: audit number
#(exchg='11' or exchg ='12') listed at NYSE or AMEX???

funda=Conn.raw_sql('''
                  select gvkey, fyear, datadate, IB, oancf, at, LT, REVT, PPEGT, CSHO, AU, 
                  oiadp, act, lct, che, dlc, txp, dp, sich
                  from compa.funda
                  where 
                    (indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C')
                    and fyear <= 1993 
                    and fyear >= 1988
                            ''',date_cols=['datadate'])

funda.rename(columns={'datadate':'date'},inplace=True)
funda['fyear']=funda['fyear'].astype(int)
funda['gvkey']=funda['gvkey'].astype(int)
funda.dropna(subset=['au'],inplace=True)
funda['au']=funda['au'].astype(str).astype(int)
funda['date']=funda['date']+MonthEnd(0)
#comp.dropna(inplace=True) don't do it!!! not or stock has sich but some of them have sic
funda.describe()


,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sich
count,46635.000000,46635.000000,46362.000000,42561.000000,46612.000000,46577.000000,46358.000000,45579.000000,46105.000000,46635.000000,46356.000000,41899.000000,43074.000000,46570.000000,46456.000000,46070.000000,45753.000000,45958.000000
mean,12498.416168,1990.572360,34.123323,89.303370,1335.377051,969.399676,897.603428,802.562676,26.476433,5.560395,94.480187,301.262396,260.864664,120.107016,196.924306,14.818368,50.401888,4492.969994
std,9573.749648,1.728956,223.233003,527.238962,7409.307060,6250.906920,4529.125007,4530.271809,86.634850,3.997733,492.823927,1635.031994,1990.835504,1177.783333,2697.493150,108.358740,298.873210,2029.912820
min,1003.000000,1988.000000,-7987.000000,-17107.322000,0.000000,0.000000,-31.503000,0.000000,0.000000,0.000000,-3449.000000,-0.262000,-0.002000,-34.000000,-0.069000,-178.605000,-0.212000,100.000000
25%,6305.000000,1989.000000,-0.923750,-0.202000,10.185000,3.972000,7.918250,2.848000,3.310000,3.000000,-0.248000,4.339000,2.321250,0.459000,0.118000,0.000000,0.300000,3081.000000
50%,11435.000000,1991.000000,0.808000,2.290000,57.712500,27.201000,55.377000,21.612000,7.499000,5.000000,2.985500,24.194000,12.307500,3.121000,1.544000,0.000000,2.009000,3845.000000
75%,15083.000000,1992.000000,10.331750,22.406000,343.382250,206.692000,332.242750,165.591000,18.333000,7.000000,25.698000,120.391500,68.946250,18.591750,12.648250,1.093750,13.055000,6036.000000
max,220508.000000,1993.000000,6539.000000,21237.000000,251506.000000,224026.000000,172913.000000,269151.000000,3213.690000,27.000000,15305.000000,56276.214000,155729.000000,78006.000000,156786.000000,5671.400000,16677.000000,9995.000000


In [4]:
funda

,gvkey,fyear,date,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sich
0,1003,1988,1989-01-31,-7.838,-2.100,16.280,16.474,32.808,2.593,2.683,5,-4.372,14.656,16.370,0.302,7.626,0.000,0.692,5712.0
1,1004,1988,1989-05-31,24.814,0.094,356.391,184.811,406.360,85.414,16.009,6,44.615,257.048,128.104,4.565,70.271,2.954,6.436,5080.0
2,1009,1988,1988-10-31,1.266,1.654,16.217,12.438,28.419,15.616,0.842,4,2.831,6.692,5.898,0.387,1.353,0.000,0.992,3460.0
3,1010,1988,1988-12-31,15.062,-34.373,1988.285,1869.021,287.723,965.852,0.015,20,90.416,793.146,658.598,661.392,64.845,4.225,48.575,3743.0
4,1011,1988,1988-12-31,0.292,0.695,4.982,1.974,2.589,5.615,3.299,6,0.072,0.647,0.613,0.054,0.014,0.000,0.396,4841.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60382,119053,1993,1993-12-31,-6.095,0.314,542.052,201.760,270.371,230.915,24.242,3,44.062,156.376,56.771,57.539,0.508,8.393,10.857,2836.0
60388,124014,1993,1994-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3572.0
60391,143972,1993,1993-12-31,104.182,151.270,2514.529,892.864,1144.714,525.787,59.628,1,198.232,321.984,308.979,6.168,8.543,0.000,79.921,4813.0
60394,154394,1993,1994-01-31,-95.300,NaN,1302.000,1009.700,6207.200,798.100,86.624,9,93.000,521.100,711.000,40.900,166.600,19.400,64.600,5411.0


In [5]:
###load industry info
Ind = Conn.raw_sql("""
                      select gvkey, sic
                      from compa.company
                     """)
Ind['gvkey']=Ind['gvkey'].astype(int)
###merge it with compustat
funda = pd.merge(funda,Ind,on=['gvkey'])

###combine sich and sic 
funda['sic'] = np.where(funda['sich']>0, funda['sich'], funda['sic'])
funda['sic']=funda['sic'].astype(int)
funda['TwoDigSIC'] = funda['sic'].astype(str).str[:2].astype(int)
funda = funda.drop(['sich'],axis=1)
funda.dropna(inplace=True)
funda.describe()

,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sic,TwoDigSIC
count,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000,37394.000000
mean,12197.453442,1990.607638,30.273417,80.553546,884.645196,557.278963,811.704323,762.463220,25.119178,5.522036,72.643941,287.494196,219.808824,61.958139,58.688732,14.089167,45.715649,4395.315024,43.661577
std,9013.085229,1.711144,205.539680,450.154603,4189.522101,2633.039344,4106.035869,4548.748283,83.369210,3.980559,343.964593,1607.316848,1284.294509,497.434841,494.112759,96.302727,281.143784,1971.252051,19.582274
min,1003.000000,1988.000000,-7987.000000,-4243.000000,0.000000,0.000000,-31.503000,0.000000,0.000000,0.000000,-3449.000000,-0.004000,0.000000,-34.000000,-0.069000,-33.226000,-0.212000,100.000000,10.000000
25%,6129.000000,1989.000000,-0.951000,-0.223000,8.873000,3.737000,8.205000,3.129000,3.249000,3.000000,-0.302000,4.508000,2.412250,0.469250,0.140000,0.000000,0.332000,3081.000000,30.000000
50%,10987.000000,1991.000000,0.668000,2.005500,48.443500,22.302000,54.645000,20.717500,7.126000,5.000000,2.439500,23.679000,11.918500,2.987000,1.369000,0.000000,1.975000,3826.000000,38.000000
75%,14888.000000,1992.000000,8.789750,20.490750,286.577000,164.086000,317.780000,153.844750,17.610000,7.000000,22.254500,114.999250,64.364000,17.128000,9.332750,1.303000,12.186000,5411.000000,54.000000
max,145348.000000,1993.000000,6539.000000,21237.000000,137338.000000,89049.000000,172913.000000,269151.000000,3213.690000,27.000000,12870.000000,56276.214000,47132.012000,21164.062000,25583.098000,3906.000000,16677.000000,9995.000000,99.000000


In [6]:
### remove obs from financial and utility
funda = funda[funda['sic'].notnull()]

funda.loc[(funda['sic'] >= 4000) & (funda['sic'] <= 4999), 'utility'] = 1

funda.loc[(funda['sic'] >= 6000) & (funda['sic'] <= 6999), 'financial'] = 1

funda = funda[funda['utility'] != 1]
funda = funda[funda['financial'] != 1]
funda=funda.drop(['utility','financial'],axis=1)
funda.describe()

,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sic,TwoDigSIC
count,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.00000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000
mean,12230.462225,1990.618831,24.572202,61.657292,695.816600,429.398261,778.63707,509.934769,23.737432,5.606587,57.864599,286.821724,200.521765,61.773248,53.165763,12.672182,34.958462,4255.453524,42.267084
std,8717.411166,1.711901,198.998206,375.085160,3849.663103,2383.162529,4243.39418,3411.638304,83.697568,4.012403,314.152002,1689.803068,1315.877663,527.140832,513.038634,97.681695,216.460322,2088.264056,20.749839
min,1003.000000,1988.000000,-7987.000000,-4243.000000,0.000000,0.000000,-1.63200,0.000000,0.000000,0.000000,-3449.000000,-0.004000,0.000000,-34.000000,-0.069000,-33.226000,-0.212000,100.000000,10.000000
25%,6190.250000,1989.000000,-1.085000,-0.314000,7.926000,3.361500,7.61325,2.867000,3.250000,3.000000,-0.396000,4.247750,2.218000,0.432000,0.122000,0.000000,0.307000,2836.000000,28.000000
50%,11113.000000,1991.000000,0.456500,1.445500,40.693000,17.741500,47.31250,16.489000,6.966000,5.000000,1.785500,21.800500,10.169000,2.743000,1.133000,0.000000,1.643000,3663.000000,36.000000
75%,14907.000000,1992.000000,6.431750,14.740250,213.751000,114.917500,267.65575,108.373500,16.437750,7.000000,16.675000,102.595750,50.912500,16.059500,7.105000,0.896000,9.279250,5500.000000,55.000000
max,119053.000000,1993.000000,6539.000000,12437.000000,106431.000000,66566.251000,172913.00000,118517.000000,3213.690000,27.000000,12870.000000,56276.214000,47132.012000,21164.062000,25583.098000,3906.000000,9221.000000,9995.000000,99.000000


In [7]:
#generate Accural

#oiadp: Operating Income After Depreciation
#act: Current Assets - Total
#che: cash/cash equivalents
#lct: Current Liabilities - Total
#dlc: Debt in Current Liabilities
#txp: Income Taxes Payable
#dp: Depreciation and Amortization
#at: Assets - Total

###generate lag 
funda_lag1 = funda[['gvkey','fyear','at','act','che','lct','dlc','txp','revt','csho','au']].copy() 

funda_lag1['fyear'] = funda_lag1['fyear'] + 1

funda_lag1 = funda_lag1.rename(columns={'at':'lag_at','act':'lag_act','che':'lag_che','lct':'lag_lct', \
                                        'dlc':'lag_dlc','txp':'lag_txp', 'revt': 'lag_revt', 'csho':'lag_csho',\
                                        'au':'lag_au'})

###generate forward
funda_forward1 = funda[['gvkey','fyear','au']].copy()

funda_forward1 = funda_forward1.rename(columns={'au':'forward_au'})

funda_forward1['fyear'] = funda_forward1['fyear'] -1 

In [8]:
funda = pd.merge(funda,funda_lag1, how='left', on=['gvkey','fyear'])
funda = pd.merge(funda,funda_forward1, how='left', on=['gvkey','fyear'])
funda.describe()

,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sic,TwoDigSIC,lag_at,lag_act,lag_che,lag_lct,lag_dlc,lag_txp,lag_revt,lag_csho,lag_au,forward_au
count,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.00000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,31608.000000,23712.000000,23712.000000,23712.000000,23712.000000,23712.000000,23712.000000,23712.000000,23712.000000,23712.000000,23712.000000
mean,12230.462225,1990.618831,24.572202,61.657292,695.816600,429.398261,778.63707,509.934769,23.737432,5.606587,57.864599,286.821724,200.521765,61.773248,53.165763,12.672182,34.958462,4255.453524,42.267084,715.859251,295.067744,62.318837,206.203551,54.848699,13.687773,806.703360,23.493077,5.618295,5.527117
std,8717.411166,1.711901,198.998206,375.085160,3849.663103,2383.162529,4243.39418,3411.638304,83.697568,4.012403,314.152002,1689.803068,1315.877663,527.140832,513.038634,97.681695,216.460322,2088.264056,20.749839,3888.899353,1694.028956,530.821780,1325.264408,515.786155,104.373981,4284.062172,82.879192,4.036171,3.914713
min,1003.000000,1988.000000,-7987.000000,-4243.000000,0.000000,0.000000,-1.63200,0.000000,0.000000,0.000000,-3449.000000,-0.004000,0.000000,-34.000000,-0.069000,-33.226000,-0.212000,100.000000,10.000000,0.000000,-0.004000,-5.884000,0.000000,-0.069000,-33.226000,-1.632000,0.000000,0.000000,0.000000
25%,6190.250000,1989.000000,-1.085000,-0.314000,7.926000,3.361500,7.61325,2.867000,3.250000,3.000000,-0.396000,4.247750,2.218000,0.432000,0.122000,0.000000,0.307000,2836.000000,28.000000,8.082000,4.367750,0.463000,2.239000,0.133000,0.000000,8.073750,3.177750,3.000000,3.000000
50%,11113.000000,1991.000000,0.456500,1.445500,40.693000,17.741500,47.31250,16.489000,6.966000,5.000000,1.785500,21.800500,10.169000,2.743000,1.133000,0.000000,1.643000,3663.000000,36.000000,41.767000,22.537000,2.796500,10.386500,1.194000,0.000000,49.809500,6.799500,5.000000,5.000000
75%,14907.000000,1992.000000,6.431750,14.740250,213.751000,114.917500,267.65575,108.373500,16.437750,7.000000,16.675000,102.595750,50.912500,16.059500,7.105000,0.896000,9.279250,5500.000000,55.000000,222.514000,107.341000,16.292500,52.749000,7.534000,1.034250,278.299250,16.353750,7.000000,7.000000
max,119053.000000,1993.000000,6539.000000,12437.000000,106431.000000,66566.251000,172913.00000,118517.000000,3213.690000,27.000000,12870.000000,56276.214000,47132.012000,21164.062000,25583.098000,3906.000000,9221.000000,9995.000000,99.000000,106431.000000,49975.512000,21164.062000,47132.012000,25583.098000,3906.000000,149185.000000,3212.132000,27.000000,27.000000


In [9]:
### use method sloan TAR 1996
funda['accrual']= ((funda['act']-funda['lag_act'])-(funda['che']-funda['lag_che']))\
                    -((funda['lct']-funda['lag_lct'])-(funda['dlc']-funda['lag_dlc'])-(funda['txp']-funda['lag_txp'])) \
                    -funda['dp']
###drop firm-year which has 0 total asset
funda=funda[funda['lag_at']!=0]
funda=funda[funda['at']!=0]
funda=funda[funda['lag_csho']!=0]
#standardize by total asset
funda['adj_earnings']=funda['oiadp']/funda['lag_at']
funda['adj_accrual']=funda['accrual']/funda['lag_at']
funda['adj_cashflow']=(funda['oiadp']/funda['lag_at']-funda['adj_accrual'])


In [10]:
funda.dropna(inplace=True)
funda.describe()

,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sic,TwoDigSIC,lag_at,lag_act,lag_che,lag_lct,lag_dlc,lag_txp,lag_revt,lag_csho,lag_au,forward_au,accrual,adj_earnings,adj_accrual,adj_cashflow
count,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.00000,16968.000000,16968.000000,16968.000000,1.696800e+04
mean,10696.068953,1990.564356,28.482445,69.431534,774.701431,472.418087,878.170522,571.310661,24.559596,5.542786,66.394125,317.757789,223.561780,66.681961,59.677847,14.578309,39.087283,4202.155941,41.729903,730.126097,302.533253,63.466676,209.583402,54.269103,14.840634,830.731197,23.277290,5.650990,5.45297,-35.909992,-0.029041,0.022633,-5.167345e-02
std,7220.482072,1.114205,213.072390,397.150346,4140.508818,2530.477188,4577.908124,3695.176419,84.198769,3.926229,349.471918,1798.229817,1421.036604,560.084209,551.276966,109.811527,232.472601,2048.709589,20.361363,3913.396769,1697.914968,534.599968,1315.933942,493.443095,112.139438,4330.268702,80.924574,4.086969,3.79268,252.362679,1.168315,5.541255,6.120453e+00
min,1004.000000,1989.000000,-6865.000000,-1006.100000,0.001000,0.000000,-1.632000,0.000000,0.000000,0.000000,-811.000000,-0.004000,0.000000,-5.884000,-0.069000,-33.226000,0.000000,100.000000,10.000000,0.001000,0.000000,-5.884000,0.000000,-0.069000,-30.638000,-1.632000,0.001000,0.000000,0.00000,-12828.000000,-64.000000,-63.500000,-7.280000e+02
25%,5757.000000,1990.000000,-0.897250,-0.129000,9.685750,3.838250,9.468000,3.605750,3.565000,3.000000,-0.264000,5.124500,2.518750,0.522000,0.132000,0.000000,0.377000,2836.000000,28.000000,8.299750,4.498500,0.498750,2.262750,0.145000,0.000000,8.546500,3.225000,3.000000,3.00000,-9.638250,-0.030614,-0.100256,1.006140e-16
50%,10200.000000,1991.000000,0.603000,2.074500,47.750500,20.651000,56.907000,20.888500,7.518000,5.000000,2.228000,25.507500,11.589000,3.271000,1.229500,0.000000,1.976000,3663.000000,36.000000,43.117500,23.086500,2.848000,10.703000,1.245500,0.005000,52.131500,6.757500,5.000000,5.00000,-1.045000,0.063652,-0.045177,1.046769e-01
75%,13745.500000,1992.000000,7.624250,18.101500,248.504750,130.986000,307.961750,129.552000,17.831000,7.000000,19.051000,118.357750,57.395500,18.629500,7.940250,1.164500,10.861250,5331.000000,54.000000,228.961250,111.706250,16.413000,53.927250,7.793250,1.174750,287.788250,16.603750,7.000000,7.00000,0.164000,0.135620,0.013779,1.836422e-01
max,119053.000000,1992.000000,6539.000000,12238.000000,106431.000000,65259.485000,149185.000000,118517.000000,3212.132000,27.000000,12870.000000,49975.512000,47132.012000,21164.062000,25583.098000,3906.000000,9221.000000,9995.000000,99.000000,106431.000000,49975.512000,21164.062000,47132.012000,25583.098000,3906.000000,147848.000000,3208.828000,27.000000,27.00000,1710.000000,39.111111,664.000000,3.550000e+01


In [11]:
###generate big 6
#The Big Six, ranked by revenue, are
#Andersen Worldwide, Ernst & Young, Deloitte & Touche, KPMG Peat Marwick, Coopers & Lybrand and Price Waterhouse.
#01 Arthur Andersen
#02 Arthur Young (prior to October 1, 1989)
#03 Coopers & Lybrand (known as Coopers & Lybrand Deloitte in the United Kingdom since April 29, 1990; Coopers & Lybrand merged with Price Waterhouse on July 1, 1998)
#04 Ernst & Young (Ernst & Whinney from July 1, 1989 to September 29, 1989; Ernst & Ernst prior to July 1, 1989)
#05 Deloitte & Touche (Deloitte, Haskins and Sells prior to December 4, 1989; Haskins & Sells prior to May 1, 1978)
#06 KPMG (Peat, Marwick, Mitchell prior to April 1, 1987)
#07 PricewaterhouseCoopers (Price Waterhouse prior to July 1, 1998 merger with Coopers and Lybrand)
#08 Touche Ross (merged with Deloitte, Haskins and Sells on December 4, 1989)
funda['big_6']=1
funda.loc[(funda['au']==1) | (funda['au']==3) | (funda['au']==4) | \
          (funda['au']==5) | (funda['au']==6) | (funda['au']==7), 'big_6']=0
funda=funda[funda['au']!=0]
funda.describe()

,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sic,TwoDigSIC,lag_at,lag_act,lag_che,lag_lct,lag_dlc,lag_txp,lag_revt,lag_csho,lag_au,forward_au,accrual,adj_earnings,adj_accrual,adj_cashflow,big_6
count,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000
mean,10701.461602,1990.563501,28.687809,69.914518,780.009819,475.627333,884.230613,575.206720,24.607374,5.581602,66.855533,319.950015,225.087344,67.145474,60.077978,14.680266,39.354813,4203.286528,41.742018,735.121305,304.616914,63.906984,211.010238,54.632194,14.944450,836.446424,23.301755,5.667715,5.476320,-36.155708,-0.021363,-0.017010,-0.004354,0.183323
std,7236.419802,1.114400,213.802594,398.496296,4154.488194,2539.024882,4593.331984,3707.791639,84.387677,3.912360,350.648768,1804.322980,1425.885552,562.014492,553.182421,110.188598,233.262782,2044.939604,20.323004,3926.613077,1703.665903,536.442651,1320.421939,495.148450,112.524543,4344.860698,81.086034,4.082148,3.779967,253.227351,1.042075,2.031968,2.369327,0.386943
min,1004.000000,1989.000000,-6865.000000,-1006.100000,0.001000,0.000000,-1.632000,0.000000,0.000000,1.000000,-811.000000,0.000000,0.000000,-5.884000,-0.069000,-33.226000,0.000000,100.000000,10.000000,0.001000,0.000000,-5.884000,0.000000,-0.069000,-30.638000,-1.632000,0.001000,0.000000,0.000000,-12828.000000,-60.263158,-63.500000,-281.466667,0.000000
25%,5729.000000,1990.000000,-0.894750,-0.129000,9.958250,3.912000,9.872250,3.701500,3.570000,3.000000,-0.251750,5.322000,2.565000,0.542000,0.133000,0.000000,0.391000,2836.000000,28.000000,8.702500,4.653500,0.515000,2.311000,0.146000,0.000000,8.806000,3.227000,3.000000,3.000000,-9.803250,-0.028078,-0.099860,0.000896,0.000000
50%,10200.000000,1991.000000,0.625500,2.152000,48.408000,21.075500,58.188000,21.415000,7.536500,5.000000,2.299000,26.089500,11.780500,3.333500,1.242500,0.001000,2.013500,3663.000000,36.000000,44.072500,23.452500,2.918000,10.886000,1.258000,0.007000,53.135500,6.777000,5.000000,5.000000,-1.065000,0.064342,-0.045036,0.105498,0.000000
75%,13756.000000,1992.000000,7.720750,18.472250,251.242000,132.708250,311.732000,130.938500,17.859250,7.000000,19.334000,119.684000,58.046250,18.837500,8.005000,1.192750,10.948500,5331.000000,54.000000,231.839750,113.070250,16.627000,54.678000,7.905750,1.202000,292.181250,16.674250,7.000000,7.000000,0.166750,0.135995,0.013782,0.183885,0.000000
max,119053.000000,1992.000000,6539.000000,12238.000000,106431.000000,65259.485000,149185.000000,118517.000000,3212.132000,27.000000,12870.000000,49975.512000,47132.012000,21164.062000,25583.098000,3906.000000,9221.000000,9995.000000,99.000000,106431.000000,49975.512000,21164.062000,47132.012000,25583.098000,3906.000000,147848.000000,3208.828000,27.000000,27.000000,1710.000000,39.111111,237.866667,24.000000,1.000000


In [12]:
funda.describe()

,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sic,TwoDigSIC,lag_at,lag_act,lag_che,lag_lct,lag_dlc,lag_txp,lag_revt,lag_csho,lag_au,forward_au,accrual,adj_earnings,adj_accrual,adj_cashflow,big_6
count,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000,16850.000000
mean,10701.461602,1990.563501,28.687809,69.914518,780.009819,475.627333,884.230613,575.206720,24.607374,5.581602,66.855533,319.950015,225.087344,67.145474,60.077978,14.680266,39.354813,4203.286528,41.742018,735.121305,304.616914,63.906984,211.010238,54.632194,14.944450,836.446424,23.301755,5.667715,5.476320,-36.155708,-0.021363,-0.017010,-0.004354,0.183323
std,7236.419802,1.114400,213.802594,398.496296,4154.488194,2539.024882,4593.331984,3707.791639,84.387677,3.912360,350.648768,1804.322980,1425.885552,562.014492,553.182421,110.188598,233.262782,2044.939604,20.323004,3926.613077,1703.665903,536.442651,1320.421939,495.148450,112.524543,4344.860698,81.086034,4.082148,3.779967,253.227351,1.042075,2.031968,2.369327,0.386943
min,1004.000000,1989.000000,-6865.000000,-1006.100000,0.001000,0.000000,-1.632000,0.000000,0.000000,1.000000,-811.000000,0.000000,0.000000,-5.884000,-0.069000,-33.226000,0.000000,100.000000,10.000000,0.001000,0.000000,-5.884000,0.000000,-0.069000,-30.638000,-1.632000,0.001000,0.000000,0.000000,-12828.000000,-60.263158,-63.500000,-281.466667,0.000000
25%,5729.000000,1990.000000,-0.894750,-0.129000,9.958250,3.912000,9.872250,3.701500,3.570000,3.000000,-0.251750,5.322000,2.565000,0.542000,0.133000,0.000000,0.391000,2836.000000,28.000000,8.702500,4.653500,0.515000,2.311000,0.146000,0.000000,8.806000,3.227000,3.000000,3.000000,-9.803250,-0.028078,-0.099860,0.000896,0.000000
50%,10200.000000,1991.000000,0.625500,2.152000,48.408000,21.075500,58.188000,21.415000,7.536500,5.000000,2.299000,26.089500,11.780500,3.333500,1.242500,0.001000,2.013500,3663.000000,36.000000,44.072500,23.452500,2.918000,10.886000,1.258000,0.007000,53.135500,6.777000,5.000000,5.000000,-1.065000,0.064342,-0.045036,0.105498,0.000000
75%,13756.000000,1992.000000,7.720750,18.472250,251.242000,132.708250,311.732000,130.938500,17.859250,7.000000,19.334000,119.684000,58.046250,18.837500,8.005000,1.192750,10.948500,5331.000000,54.000000,231.839750,113.070250,16.627000,54.678000,7.905750,1.202000,292.181250,16.674250,7.000000,7.000000,0.166750,0.135995,0.013782,0.183885,0.000000
max,119053.000000,1992.000000,6539.000000,12238.000000,106431.000000,65259.485000,149185.000000,118517.000000,3212.132000,27.000000,12870.000000,49975.512000,47132.012000,21164.062000,25583.098000,3906.000000,9221.000000,9995.000000,99.000000,106431.000000,49975.512000,21164.062000,47132.012000,25583.098000,3906.000000,147848.000000,3208.828000,27.000000,27.000000,1710.000000,39.111111,237.866667,24.000000,1.000000


In [13]:
### remove company that change auditor type
funda['auditor_change']=funda.groupby('gvkey')['big_6'].transform(np.nanmean)
funda=funda[(funda['auditor_change']==1) | (funda['auditor_change']==0)]

In [14]:
###remove company that have no counterpart
#create decile
for year in funda.groupby(['fyear']):
  funda['cashflow_Decile'] = pd.qcut(funda['adj_cashflow'],10,labels=[1,2,3,4,5,6,7,8,9,10])

funda['counterpart']=funda.groupby(['TwoDigSIC','fyear','cashflow_Decile'])['big_6'].transform(np.nanmean)

funda=funda[(funda['counterpart']!=1) & (funda['counterpart']!=0)]
funda.describe()

,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sic,TwoDigSIC,lag_at,lag_act,lag_che,lag_lct,lag_dlc,lag_txp,lag_revt,lag_csho,lag_au,forward_au,accrual,adj_earnings,adj_accrual,adj_cashflow,big_6,auditor_change,counterpart
count,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000
mean,10899.124003,1990.591169,22.981081,55.307062,654.901014,400.105912,700.144551,424.567973,23.677535,5.636854,54.070439,293.285693,203.481869,68.635725,60.187920,11.670349,31.709199,4159.633470,41.255580,611.141512,276.439487,64.357044,189.284967,54.074158,11.869706,660.976488,22.380739,5.656112,5.567561,-27.424171,-0.035336,-0.002737,-0.032598,0.196842,0.196842,0.196842
std,7386.535942,1.111713,167.390570,294.312143,3580.803528,2338.827015,4048.243781,2449.981002,87.184803,3.960993,265.149802,1836.422216,1431.001341,617.627665,609.591621,82.331818,191.406821,2039.471634,20.257734,3323.679246,1717.245258,585.985766,1310.116103,543.839291,87.307607,3804.561866,84.060265,4.049316,3.888702,210.691485,1.111044,2.274396,2.721651,0.397628,0.397628,0.130143
min,1004.000000,1989.000000,-6865.000000,-1006.100000,0.001000,0.000000,-1.632000,0.000000,0.000000,1.000000,-811.000000,0.000000,0.000000,-0.001000,0.000000,-33.226000,0.000000,100.000000,10.000000,0.001000,0.000000,-0.014000,0.000000,0.000000,-30.638000,-1.632000,0.001000,0.000000,0.000000,-12828.000000,-60.263158,-39.000000,-281.466667,0.000000,0.000000,0.022222
25%,5776.000000,1990.000000,-0.983000,-0.240500,8.811500,3.410000,8.078000,3.200000,3.595000,3.000000,-0.364000,4.825000,2.228500,0.522500,0.100000,0.000000,0.329000,2835.000000,28.000000,7.490500,4.151500,0.492500,2.023500,0.113000,0.000000,7.323000,3.217500,3.000000,3.000000,-7.692500,-0.044270,-0.099731,-0.018839,0.000000,0.000000,0.105263
50%,10326.000000,1991.000000,0.468000,1.560000,41.949000,17.004000,45.846000,16.596000,7.440000,5.000000,1.696000,23.191000,9.851000,3.183000,1.043000,0.000000,1.677000,3661.000000,36.000000,37.115000,20.522000,2.707000,9.074000,1.053000,0.000000,41.097000,6.665000,5.000000,5.000000,-0.796000,0.059212,-0.041554,0.095306,0.000000,0.000000,0.166667
75%,13951.000000,1992.000000,6.379500,14.635500,196.080500,96.450000,231.526000,100.499000,16.824000,7.000000,15.029000,100.045500,46.478500,18.261500,6.810000,0.913000,8.737000,5090.000000,50.000000,182.636500,93.161500,15.655000,43.728500,6.824500,0.938000,213.277000,15.477000,7.000000,7.000000,0.280000,0.135993,0.021001,0.180030,0.000000,0.000000,0.250000
max,119053.000000,1992.000000,5010.000000,10646.000000,92473.000000,65259.485000,149185.000000,107599.000000,3212.132000,27.000000,9313.000000,49975.512000,47132.012000,21164.062000,25583.098000,2755.000000,9221.000000,9995.000000,99.000000,92473.000000,49975.512000,21164.062000,47132.012000,25583.098000,3159.000000,147848.000000,3208.828000,27.000000,27.000000,1710.000000,39.111111,237.866667,24.000000,1.000000,1.000000,0.857143


In [15]:
#generate variable for regression1
funda['adj_change_revt']=((funda['revt']-funda['lag_revt'])/funda['lag_at'])
funda['adj_ppegt']=funda['ppegt']/funda['lag_at']

In [16]:
#generate variable for regression2
funda['lev']=funda['lt']/funda['at']
for year in funda.groupby(['fyear','TwoDigSIC']):
  funda['lev_decile'] = pd.qcut(funda['lev'],10,labels=[1,2,3,4,5,6,7,8,9,10])
funda.loc[funda['lev_decile']==10,'Hilev']=1
funda.loc[funda['lev_decile']<10,'Hilev']=0

funda['Absaccr']=abs(funda['adj_accrual'])

funda['share_change']=(funda['csho']-funda['lag_csho'])/funda['lag_csho']

funda.loc[funda['share_change']>=0.1,'shareincr']=1
funda.loc[funda['share_change']<0.1,'shareincr']=0

funda.loc[funda['share_change']<=-0.1,'sharedecr']=1
funda.loc[funda['share_change']>-0.1,'sharedecr']=0

###for company with obs start in 1989
funda.loc[(funda['au']!=funda['lag_au']) & (funda['fyear']==1989), 'OldAud']=1
funda.loc[(funda['au']==funda['lag_au']) & (funda['fyear']==1989), 'OldAud']=0

funda.loc[(funda['au']!=funda['forward_au']) & (funda['fyear']==1992), 'NewAud']=1
funda.loc[(funda['au']==funda['forward_au']) & (funda['fyear']==1992), 'NewAud']=0

funda['OldAud']=funda.groupby('gvkey')['OldAud'].transform(np.nanmean)
funda['NewAud']=funda.groupby('gvkey')['NewAud'].transform(np.nanmean)


###for company with obs start in 1990
funda.loc[(funda['au']!=funda['lag_au']) & (funda['OldAud'].isnull()) & (funda['fyear']==1990), 'OldAud']=1
funda.loc[(funda['au']==funda['lag_au']) & (funda['OldAud'].isnull()) & (funda['fyear']==1990), 'OldAud']=0

funda.loc[(funda['au']!=funda['forward_au']) & (funda['NewAud'].isnull()) & (funda['fyear']==1991), 'NewAud']=1
funda.loc[(funda['au']==funda['forward_au']) & (funda['NewAud'].isnull()) & (funda['fyear']==1991), 'NewAud']=0

funda['OldAud']=funda.groupby('gvkey')['OldAud'].transform(np.nanmean)
funda['NewAud']=funda.groupby('gvkey')['NewAud'].transform(np.nanmean)

###for company with obs start in 1991
funda.loc[(funda['au']!=funda['lag_au']) & (funda['OldAud'].isnull()) & (funda['fyear']==1991), 'OldAud']=1
funda.loc[(funda['au']==funda['lag_au']) & (funda['OldAud'].isnull()) & (funda['fyear']==1991), 'OldAud']=0

funda.loc[(funda['au']!=funda['forward_au']) & (funda['NewAud'].isnull()) & (funda['fyear']==1990), 'NewAud']=1
funda.loc[(funda['au']==funda['forward_au']) & (funda['NewAud'].isnull()) & (funda['fyear']==1990), 'NewAud']=0

funda['OldAud']=funda.groupby('gvkey')['OldAud'].transform(np.nanmean)
funda['NewAud']=funda.groupby('gvkey')['NewAud'].transform(np.nanmean)

###company with only 1 year obs
funda.loc[(funda['OldAud'].isnull()), 'OldAud']=0
funda.loc[(funda['NewAud'].isnull()), 'NewAud']=0

In [17]:
### log asset
funda['log_asset']=np.log(funda['at']+1)

In [18]:
### adjust income before extradinary item
funda['adj_ib']=funda['ib']/funda['at']
funda['adj_oiadp']=funda['oiadp']/funda['at']


In [19]:
funda.describe()

,gvkey,fyear,ib,oancf,at,lt,revt,ppegt,csho,au,oiadp,act,lct,che,dlc,txp,dp,sic,TwoDigSIC,lag_at,lag_act,lag_che,lag_lct,lag_dlc,lag_txp,lag_revt,lag_csho,lag_au,forward_au,accrual,adj_earnings,adj_accrual,adj_cashflow,big_6,auditor_change,counterpart,adj_change_revt,adj_ppegt,lev,Hilev,Absaccr,share_change,shareincr,sharedecr,OldAud,NewAud,log_asset,adj_ib,adj_oiadp
count,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000,12411.000000
mean,10899.124003,1990.591169,22.981081,55.307062,654.901014,400.105912,700.144551,424.567973,23.677535,5.636854,54.070439,293.285693,203.481869,68.635725,60.187920,11.670349,31.709199,4159.633470,41.255580,611.141512,276.439487,64.357044,189.284967,54.074158,11.869706,660.976488,22.380739,5.656112,5.567561,-27.424171,-0.035336,-0.002737,-0.032598,0.196842,0.196842,0.196842,0.333011,0.862708,0.590168,0.099992,0.166830,7.984717,0.247281,0.041898,0.163242,0.058819,3.934325,-0.106670,-0.049967
std,7386.535942,1.111713,167.390570,294.312143,3580.803528,2338.827015,4048.243781,2449.981002,87.184803,3.960993,265.149802,1836.422216,1431.001341,617.627665,609.591621,82.331818,191.406821,2039.471634,20.257734,3323.679246,1717.245258,585.985766,1310.116103,543.839291,87.307607,3804.561866,84.060265,4.049316,3.888702,210.691485,1.111044,2.274396,2.721651,0.397628,0.397628,0.130143,10.513139,15.302840,1.739856,0.300001,2.268270,309.830934,0.431449,0.200365,0.369601,0.235295,2.103370,1.696368,1.603827
min,1004.000000,1989.000000,-6865.000000,-1006.100000,0.001000,0.000000,-1.632000,0.000000,0.000000,1.000000,-811.000000,0.000000,0.000000,-0.001000,0.000000,-33.226000,0.000000,100.000000,10.000000,0.001000,0.000000,-0.014000,0.000000,0.000000,-30.638000,-1.632000,0.001000,0.000000,0.000000,-12828.000000,-60.263158,-39.000000,-281.466667,0.000000,0.000000,0.022222,-12.734928,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.001000,-163.000000,-160.500000
25%,5776.000000,1990.000000,-0.983000,-0.240500,8.811500,3.410000,8.078000,3.200000,3.595000,3.000000,-0.364000,4.825000,2.228500,0.522500,0.100000,0.000000,0.329000,2835.000000,28.000000,7.490500,4.151500,0.492500,2.023500,0.113000,0.000000,7.323000,3.217500,3.000000,3.000000,-7.692500,-0.044270,-0.099731,-0.018839,0.000000,0.000000,0.105263,-0.038249,0.282520,0.292868,0.000000,0.035375,0.000000,0.000000,0.000000,0.000000,0.000000,2.283555,-0.072587,-0.044141
50%,10326.000000,1991.000000,0.468000,1.560000,41.949000,17.004000,45.846000,16.596000,7.440000,5.000000,1.696000,23.191000,9.851000,3.183000,1.043000,0.000000,1.677000,3661.000000,36.000000,37.115000,20.522000,2.707000,9.074000,1.053000,0.000000,41.097000,6.665000,5.000000,5.000000,-0.796000,0.059212,-0.041554,0.095306,0.000000,0.000000,0.166667,0.069685,0.500143,0.492780,0.000000,0.073454,0.005543,0.000000,0.000000,0.000000,0.000000,3.760013,0.021035,0.055823
75%,13951.000000,1992.000000,6.379500,14.635500,196.080500,96.450000,231.526000,100.499000,16.824000,7.000000,15.029000,100.045500,46.478500,18.261500,6.810000,0.913000,8.737000,5090.000000,50.000000,182.636500,93.161500,15.655000,43.728500,6.824500,0.938000,213.277000,15.477000,7.000000,7.000000,0.280000,0.135993,0.021001,0.180030,0.000000,0.000000,0.250000,0.256115,0.824088,0.680907,0.000000,0.139273,0.096402,0.000000,0.000000,0.000000,0.000000,5.283612,0.068847,0.117825
max,119053.000000,1992.000000,5010.000000,10646.000000,92473.000000,65259.485000

In [20]:
### DAC computed by sic and year
reg= LinearRegression()
codes=funda.TwoDigSIC.unique()
years=funda.fyear.unique()
fyear=[]
TwoDigSIC=[]
intercept = []
coeff = []

for year in years:
    Data=funda[funda['fyear']==year]
    for code in codes:        
        Data1=Data[Data['TwoDigSIC']==code]       
        x=Data1[['adj_change_revt','adj_ppegt']]
        y=Data1['adj_accrual']
        if len(y)>=1:
            result=reg.fit(x,y)
            coeff.append(result.coef_)
            intercept.append(result.intercept_)
            fyear.append(year)
            TwoDigSIC.append(code)
df1=pd.DataFrame(coeff)
df1['intercept']=intercept
df1['TwoDigSIC']=TwoDigSIC
df1['fyear']=fyear
df1.columns=['adj_change_revt_coeff','adj_ppegt_coeff','intercept','TwoDigSIC','fyear']

df1.describe()

,adj_change_revt_coeff,adj_ppegt_coeff,intercept,TwoDigSIC,fyear
count,162.000000,162.000000,162.000000,162.000000,162.000000
mean,0.118921,0.235357,-0.065590,44.172840,1990.524691
std,0.424038,2.601340,0.317901,23.295984,1.126753
min,-4.062092,-7.675964,-2.771076,10.000000,1989.000000
25%,0.035544,-0.100548,-0.079791,26.000000,1990.000000
50%,0.126568,-0.036921,-0.022454,36.000000,1991.000000
75%,0.240523,0.029422,0.022607,58.000000,1992.000000
max,1.310328,29.160930,0.486690,99.000000,1992.000000


In [21]:
funda=pd.merge(funda,df1,how='left',on=['TwoDigSIC','fyear'])

In [22]:
### DAC computed by sic 
'''reg= LinearRegression()
codes=funda.TwoDigSIC.unique()
years=funda.fyear.unique()
fyear=[]
TwoDigSIC=[]
intercept = []
coeff = []

for code in codes:        
    Data=funda[funda['TwoDigSIC']==code]       
    x=Data[['adj_change_revt','adj_ppegt']]
    y=Data['adj_accrual']
    result=reg.fit(x,y)
    coeff.append(result.coef_)
    intercept.append(result.intercept_)
    fyear.append(year)
    TwoDigSIC.append(code)
    
df1=pd.DataFrame(coeff)
df1['intercept']=intercept
df1['TwoDigSIC']=TwoDigSIC
df1.columns=['adj_change_revt_coeff','adj_ppegt_coeff','intercept','TwoDigSIC']

funda=pd.merge(funda,df1,how='left',on=['TwoDigSIC'])

'''

"reg= LinearRegression()\ncodes=funda.TwoDigSIC.unique()\nyears=funda.fyear.unique()\nfyear=[]\nTwoDigSIC=[]\nintercept = []\ncoeff = []\n\nfor code in codes:        \n    Data=funda[funda['TwoDigSIC']==code]       \n    x=Data[['adj_change_revt','adj_ppegt']]\n    y=Data['adj_accrual']\n    result=reg.fit(x,y)\n    coeff.append(result.coef_)\n    intercept.append(result.intercept_)\n    fyear.append(year)\n    TwoDigSIC.append(code)\n    \ndf1=pd.DataFrame(coeff)\ndf1['intercept']=intercept\ndf1['TwoDigSIC']=TwoDigSIC\ndf1.columns=['adj_change_revt_coeff','adj_ppegt_coeff','intercept','TwoDigSIC']\n\nfunda=pd.merge(funda,df1,how='left',on=['TwoDigSIC'])\n\n"

In [23]:
funda['DAC']=funda['adj_accrual']-(funda['adj_change_revt_coeff']*funda['adj_change_revt']+\
             funda['adj_ppegt_coeff']*funda['adj_ppegt']+funda['intercept'])

In [24]:
funda[funda['big_6']==0]['DAC'].describe()

count    9968.000000
mean        0.003069
std         0.244553
min        -4.672809
25%        -0.060966
50%         0.000000
75%         0.057935
max         9.967505
Name: DAC, dtype: float64

In [25]:
funda[funda['big_6']==1]['DAC'].describe()

count    2443.000000
mean       -0.012523
std         1.067220
min       -36.228924
25%        -0.080234
50%         0.003213
75%         0.083457
max         5.182315
Name: DAC, dtype: float64

In [26]:
################################################ Descriptive ################################################ 

In [27]:
import ipystata  
from ipystata.config import config_stata  
config_stata('/Applications/Stata/StataSE.app/Contents/MacOS/stataSE')  


IPyStata is loaded in batch mode.


In [28]:
Data=funda[['gvkey','fyear','DAC','lag_at','adj_accrual','adj_change_revt','adj_ppegt', 'lev',\
            'big_6','adj_cashflow','log_asset','Hilev','adj_ib','Absaccr','shareincr','sharedecr','OldAud','NewAud','TwoDigSIC']]


In [29]:
%%stata --data Data
sum log_asset adj_ib adj_cashflow lev adj_accrual Absaccr if big_6==0
sum log_asset adj_ib adj_cashflow lev adj_accrual Absaccr if big_6==1


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
   log_asset |      9,968    4.332184    2.027045   .0178399   11.43468
      adj_ib |      9,968    -.069068    .7825144  -60.71429   .9820104
adj_cashflow |      9,968   -.0115806    2.963248  -281.4667   23.22222
         lev |      9,968    .5428977    .5888764          0   29.87308
 adj_accrual |      9,968    .0040379    2.448716  -4.327273   237.8667
-------------+---------------------------------------------------------
     Absaccr |      9,968    .1491309    2.444174   5.57e-18   237.8667

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
   log_asset |      2,443     2.31097    1.558906   .0009995   11.24717
      adj_ib |      2,443   -.2600936    3.477853       -163        4.5
adj_cashflow |      2,443    -.118354    1.339977        -37  

In [30]:
%%stata --data Data
winsor2 DAC adj_cashflow log_asset Absaccr, replace cuts(1 99)
reg DAC i.big_6 adj_cashflow log_asset i.Hilev Absaccr i.shareincr i.sharedecr i.OldAud i.NewAud 


      Source |       SS           df       MS      Number of obs   =    12,411
-------------+----------------------------------   F(9, 12401)     =    127.04
       Model |  30.7053937         9  3.41171041   Prob > F        =    0.0000
    Residual |  333.045838    12,401  .026856369   R-squared       =    0.0844
-------------+----------------------------------   Adj R-squared   =    0.0837
       Total |  363.751232    12,410  .029311139   Root MSE        =    .16388

------------------------------------------------------------------------------
         DAC | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     1.big_6 |   .0146057   .0040341     3.62   0.000     .0066982    .0225132
adj_cashflow |  -.1469366   .0045708   -32.15   0.000     -.155896   -.1379772
   log_asset |   .0070561   .0008258     8.54   0.000     .0054373    .0086748
     1.Hilev |  -.0447749   .0050027    -8.95   0.